In [1]:
# IPython candies...
from IPython.display import Image
from IPython.core.display import HTML 

Seq2Seq with PyTorch
====

Sequence-to-Sequence (Seq2Seq) learning is a useful class of neural network model to map sequential input into an output sequence. It has been shown to work well on various task, from machine translation to interpreting Python without an interpreter. {{citations-needed}}

This notebook is a hands-on session to write an encoder-decoder Seq2Seq network using PyTorch for [DataScience SG meetup](https://www.meetup.com/DataScience-SG-Singapore/events/246541733/). 

Acknowledgements
====
The materials are largely based on the 

 - [intermediate PyTorch tutorials by Sean Robertson](http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) and 
 - [Luong et al. tutorial on neural machine translation in ACL16](https://sites.google.com/site/acl16nmt/home).

The dataset used in this exercise is hosted on https://www.kaggle.com/alvations/sg-kopi

Kopi Problems
====

In this hands-on session, we want to **train a neural network to translate from Singlish Kopi orders to English?**


**"Singlish" -> English**

```
"Kopi" -> Coffee with condensed milk
"Kopi O" -> Coffee without milk or sugar
"Kopi dinosaur gau siew dai peng" -> ???
```

(Image Source: http://www.straitstimes.com/lifestyle/food/get-your-kopi-kick)

In [34]:
Image(url="https://static.straitstimes.com.sg/sites/default/files/160522_kopi.jpg", width=750)

Seriously?
====

Yes, we'll be translating Singlish Kopi orders to English using the [sequence-to-sequence network](https://arxiv.org/abs/1409.3215) {{citations-needed}}. 

But first...
====

Data Munging
====

Before any machine/deep learning, we have to get some data and "hammer" it until we get it into the shape we want.

> *Data scientists spend 60% of their time on cleaning and organizing data. Collecting data sets comes second at 19% of their time, meaning data scientists spend around 80% of their time on preparing and managing data for analysis.*

> (Source: [Gil Press](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/#3e4dc0416f63) Forbes article)

**Step 1:** Take the data from somewhere, in this case: http://kaggle.com/alvations/sg-kopi.

**Step 2:** Import your favorite dataframe and text processing library.

**Step 3:** Munge the data till desired.

In [4]:
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from nltk import word_tokenize

# Reads the tab-delimited data using Pandas.
kopitiam = pd.read_csv('coffee-culture-sg.tsv', sep='\t')
kopitiam.head()

,Local Terms,Meaning,Source
0,Kopi O,Black Coffee with Sugar,https://daneshd.com/2010/02/28/a-rough-guide-t...
1,Kopi,Black Coffee with Condensed Milk,https://daneshd.com/2010/02/28/a-rough-guide-t...
2,Kopi C,Black Coffee with Evaporated Milk,https://daneshd.com/2010/02/28/a-rough-guide-t...
3,Kopi Kosong,Black Coffee without sugar or milk,https://daneshd.com/2010/02/28/a-rough-guide-t...
4,Kopi Gah Dai,Black Coffee with extra condensed milk,https://daneshd.com/2010/02/28/a-rough-guide-t...


In [9]:
START, START_IDX = '<s>',  0
END, END_IDX = '</s>', 1

# We use this idiom to tokenize our sentences in the dataframe column:
# >>> DataFrame['column'].apply(str.lower).apply(word_tokenize)

# Also we added the START and the END symbol to the sentences. 
singlish_sents = [START] + kopitiam['Local Terms'].apply(str.lower).apply(word_tokenize) + [END]
english_sents = [START] + kopitiam['Meaning'].apply(str.lower).apply(word_tokenize) + [END]

# We're sort of getting into the data into the shape we want. 
# But now it's still too humanly readable and redundant.
## Cut-away: Computers like it to be simpler, more concise. -_-|||
print('First Singlish sentence:', singlish_sents[0])
print('First English sentence:', english_sents[0])

First Singlish sentence: ['<s>', 'kopi', 'o', '</s>']
First English sentence: ['<s>', 'black', 'coffee', 'with', 'sugar', '</s>']


In [10]:
# Let's convert the individual words into some sort of unique index 
# and use the unique to represent the words. 
## Cut-away: Integers = 1-2 bytes vs UTF-8 Strings = no. of chars * 1-2 bytes. @_@

english_vocab = Dictionary([['<s>'], ['</s>'], ['UNK']])
english_vocab.add_documents(english_sents)

singlish_vocab = Dictionary([['<s>'], ['</s>'], ['UNK']])
singlish_vocab.add_documents(singlish_sents)

# First ten words in the vocabulary.
print('First 10 Singlish words:\n', sorted(singlish_vocab.items())[:10])
print()
print('First 10 English words:\n', sorted(english_vocab.items())[:10])

First 10 Singlish words:
 [(0, '<s>'), (1, '</s>'), (2, 'UNK'), (3, 'kopi'), (4, 'o'), (5, 'c'), (6, 'kosong'), (7, 'dai'), (8, 'gah'), (9, 'siew')]

First 10 English words:
 [(0, '<s>'), (1, '</s>'), (2, 'UNK'), (3, 'black'), (4, 'coffee'), (5, 'sugar'), (6, 'with'), (7, 'condensed'), (8, 'milk'), (9, 'evaporated')]


In [12]:
# Now, convert all the sentences into list of the indices 
print('First Singlish sentence:', singlish_vocab.doc2idx(singlish_sents[0]) )
print('First English sentence:', english_vocab.doc2idx(english_sents[0]) )

First Singlish sentence: [0, 3, 4, 1]
First English sentence: [0, 3, 4, 6, 5, 1]


Torch Refresher
====


In [58]:
import torch
# Generate a 3x4 matrix.
x = torch.rand(3,4) 
x


 0.8729  0.1330  0.1536  0.2883
 0.0354  0.5267  0.4375  0.2055
 0.3164  0.0791  0.0782  0.2595
[torch.FloatTensor of size 3x4]

In [60]:
# Reshaping the matrix into a 2x6 matrix.
x.view(2,6)


 0.8729  0.1330  0.1536  0.2883  0.0354  0.5267
 0.4375  0.2055  0.3164  0.0791  0.0782  0.2595
[torch.FloatTensor of size 2x6]

In [62]:
# Also, reshaping the matrix into a 2x6 matrix.
# But now we specify the no. row and 
# let torch guess no. of columns by specifying -1.
x.view(2,-1)


 0.8729  0.1330  0.1536  0.2883  0.0354  0.5267
 0.4375  0.2055  0.3164  0.0791  0.0782  0.2595
[torch.FloatTensor of size 2x6]

In [65]:
# Reshape the 3x4 matrix into a 2x3x? matrix.
# What's ? in this case?
x.view(2, 3 , -1)


(0 ,.,.) = 
  0.8729  0.1330
  0.1536  0.2883
  0.0354  0.5267

(1 ,.,.) = 
  0.4375  0.2055
  0.3164  0.0791
  0.0782  0.2595
[torch.FloatTensor of size 2x3x2]

In [67]:
x.view(1, 1, -1)


(0 ,.,.) = 

Columns 0 to 8 
   0.8729  0.1330  0.1536  0.2883  0.0354  0.5267  0.4375  0.2055  0.3164

Columns 9 to 11 
   0.0791  0.0782  0.2595
[torch.FloatTensor of size 1x1x12]

The Seq2Seq Model
====

A Recurrent Neural Network (RNN), is a network that operates on a sequence and uses its own output as input for subsequent steps.

> *The general idea is to make **two recurrent neural network transform from one sequence to another**. An encoder network condenses an input sequence into a vector and a decoder netwrok unfolds the vector into a new sequence.*



In [39]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()



 0.2025  0.6509  0.4779  0.4420
 0.2349  0.1570  0.3148  0.5949
 0.7355  0.9863  0.8563  0.0212
[torch.FloatTensor of size 3x4]

In [47]:
import torch 
import numpy as np

a = torch.range(1, 12)

np.array(a.view(4, -1))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.3. Note that arange generates values in [start; end), not [start; end].
  after removing the cwd from sys.path.


array([[ 1.,  2.,  3.],
       [ 4.,  5.,  6.],
       [ 7.,  8.,  9.],
       [10., 11., 12.]], dtype=float32)

The Encoder
====

The encoder of a seq2seq network is a RNN that outputs some value for every word from the input sentence. For every input word the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word.


<img src="http://pytorch.org/tutorials/_images/encoder-network.png" align='left'>

In [68]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result


Simple Decoder
====

In the simplest seq2seq decoder we use only last output of the encoder. This last output is sometimes called the context vector as it encodes context from the entire sequence. This context vector is used as the initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and hidden state. The initial input token is the start-of-string `<s>` token, and the first hidden state is the context vector (the encoder’s last hidden state).


<img src="http://pytorch.org/tutorials/_images/decoder-network.png" align='left'>


In [69]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

Training the Model
====

To train we run the input sentence through the encoder, and keep track of every output and the latest hidden state. Then the decoder is given the <SOS> token as its first input, and the last hidden state of the encoder as its first hidden state.

“Teacher forcing” is the concept of using the real target outputs as each next input, instead of using the decoder’s guess as the next input. Using teacher forcing causes it to converge faster but when the trained network is exploited, it may exhibit instability.

You can observe outputs of teacher-forced networks that read with coherent grammar but wander far from the correct translation - intuitively it has learned to represent the output grammar and can “pick up” the meaning once the teacher tells it the first few words, but it has not properly learned how to create the sentence from the translation in the first place.

Because of the freedom PyTorch’s autograd gives us, we can randomly choose to use teacher forcing or not with a simple if statement. Turn teacher_forcing_ratio up to use more of it.



In [70]:
teacher_forcing_ratio = 0.5


def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]

            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length


NameError: name 'MAX_LENGTH' is not defined

In [9]:
START, START_IDX = '<s>',  0
END, END_IDX = '</s>', 1

English = [START] + kopitiam['Meaning'].apply(str.lower).apply(word_tokenize) + [END]
Singlish = [START] + kopitiam['Local Terms'].apply(str.lower).apply(word_tokenize) + [END]



In [45]:
english_vocab = Dictionary([['<s>'], ['</s>'], ['UNK']])
english_vocab.add_documents(Singlish)
[english_vocab.doc2idx(sent) for sent in Singlish]


[[0, 3, 4, 1],
 [0, 3, 1],
 [0, 3, 5, 1],
 [0, 3, 6, 1],
 [0, 3, 8, 7, 1],
 [0, 3, 9, 7, 1],
 [0, 3, 4, 9, 7, 1],
 [0, 3, 10, 1],
 [0, 3, 4, 10, 1],
 [0, 3, 11, 1],
 [0, 3, 4, 11, 1],
 [0, 3, 12, 1],
 [0, 3, 4, 12, 1],
 [0, 3, 11, 12, 1],
 [0, 3, 4, 11, 12, 1],
 [0, 3, 6, 12, 1],
 [0, 3, 11, 6, 12, 1],
 [0, 3, 1],
 [0, 3, 9, 7, 1],
 [0, 3, 13, 7, 1],
 [0, 3, 10, 1],
 [0, 3, 11, 1],
 [0, 3, 10, 9, 7, 1],
 [0, 3, 10, 13, 7, 1],
 [0, 3, 11, 9, 7, 1],
 [0, 3, 11, 13, 7, 1],
 [0, 3, 6, 1],
 [0, 3, 5, 1],
 [0, 3, 5, 9, 7, 1],
 [0, 3, 5, 13, 7, 1],
 [0, 3, 5, 10, 1],
 [0, 3, 5, 11, 1],
 [0, 3, 5, 10, 9, 7, 1],
 [0, 3, 5, 10, 13, 7, 1],
 [0, 3, 5, 11, 9, 7, 1],
 [0, 3, 5, 11, 13, 7, 1],
 [0, 3, 5, 6, 1],
 [0, 3, 4, 1],
 [0, 3, 4, 9, 7, 1],
 [0, 3, 4, 13, 7, 1],
 [0, 3, 4, 10, 1],
 [0, 3, 4, 11, 1],
 [0, 3, 4, 10, 9, 7, 1],
 [0, 3, 4, 10, 13, 7, 1],
 [0, 3, 4, 11, 9, 7, 1],
 [0, 3, 4, 11, 13, 7, 1],
 [0, 3, 4, 6, 1],
 [0, 14, 1],
 [0, 14, 9, 7, 1],
 [0, 14, 13, 7, 1],
 [0, 14, 6, 1],
 [0, 14, 1

In [87]:

class Corpus:
    def __init__(self, name, documents):
        """
        :param name: The name of the corpus. 
        :type name: str
        :param documents: The document stream of the corpus, 
                          outer list contains sentences,
                          inner list contains tokens.
        :type documents: iter(list(str))
        """
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {START_IDX: START, END_IDX: END}
        self.n_words = 2 # Count START and END tokens.
        self.populate_corpus(documents)
        
    
    
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {START_IDX: START, END_IDX: END}
        self.n_words = 2  # Count START and END tokens.

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...


FileNotFoundError: [Errno 2] No such file or directory: 'data/eng-fra.txt'

In [4]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [19]:
x = "C O Gah Kosong Siew Dai Po Gau Peng None".split()

In [54]:
from itertools import product
x = "C O Gah Kosong Siew Dai Po Gau Peng None".split()
for i, j in product(x, x):
    i = '' if i == 'None' else i
    j = '' if j == 'None' else j
    y = i + ' ' + j
    y = y.strip()
    print(y)
    for k, l in product(x, [y]):
        k = '' if k == 'None' else k
        l = '' if l == 'None' else l
        z = k + ' ' + l
        z = z.strip()
        print(z)
        for m, n in product(x, [z]):
            m = '' if m == 'None' else m
            n = '' if n == 'None' else n
            zz = m + ' ' + n
            zz = zz.strip()
            print(zz)


C C
C C C
C C C C
O C C C
Gah C C C
Kosong C C C
Siew C C C
Dai C C C
Po C C C
Gau C C C
Peng C C C
C C C
O C C
C O C C
O O C C
Gah O C C
Kosong O C C
Siew O C C
Dai O C C
Po O C C
Gau O C C
Peng O C C
O C C
Gah C C
C Gah C C
O Gah C C
Gah Gah C C
Kosong Gah C C
Siew Gah C C
Dai Gah C C
Po Gah C C
Gau Gah C C
Peng Gah C C
Gah C C
Kosong C C
C Kosong C C
O Kosong C C
Gah Kosong C C
Kosong Kosong C C
Siew Kosong C C
Dai Kosong C C
Po Kosong C C
Gau Kosong C C
Peng Kosong C C
Kosong C C
Siew C C
C Siew C C
O Siew C C
Gah Siew C C
Kosong Siew C C
Siew Siew C C
Dai Siew C C
Po Siew C C
Gau Siew C C
Peng Siew C C
Siew C C
Dai C C
C Dai C C
O Dai C C
Gah Dai C C
Kosong Dai C C
Siew Dai C C
Dai Dai C C
Po Dai C C
Gau Dai C C
Peng Dai C C
Dai C C
Po C C
C Po C C
O Po C C
Gah Po C C
Kosong Po C C
Siew Po C C
Dai Po C C
Po Po C C
Gau Po C C
Peng Po C C
Po C C
Gau C C
C Gau C C
O Gau C C
Gah Gau C C
Kosong Gau C C
Siew Gau C C
Dai Gau C C
Po Gau C C
Gau Gau C C
Peng Gau C C
Gau C C
Peng C C
C Peng

Siew Peng O Gah
Dai Peng O Gah
Po Peng O Gah
Gau Peng O Gah
Peng Peng O Gah
Peng O Gah
O Gah
C O Gah
O O Gah
Gah O Gah
Kosong O Gah
Siew O Gah
Dai O Gah
Po O Gah
Gau O Gah
Peng O Gah
O Gah
O Kosong
C O Kosong
C C O Kosong
O C O Kosong
Gah C O Kosong
Kosong C O Kosong
Siew C O Kosong
Dai C O Kosong
Po C O Kosong
Gau C O Kosong
Peng C O Kosong
C O Kosong
O O Kosong
C O O Kosong
O O O Kosong
Gah O O Kosong
Kosong O O Kosong
Siew O O Kosong
Dai O O Kosong
Po O O Kosong
Gau O O Kosong
Peng O O Kosong
O O Kosong
Gah O Kosong
C Gah O Kosong
O Gah O Kosong
Gah Gah O Kosong
Kosong Gah O Kosong
Siew Gah O Kosong
Dai Gah O Kosong
Po Gah O Kosong
Gau Gah O Kosong
Peng Gah O Kosong
Gah O Kosong
Kosong O Kosong
C Kosong O Kosong
O Kosong O Kosong
Gah Kosong O Kosong
Kosong Kosong O Kosong
Siew Kosong O Kosong
Dai Kosong O Kosong
Po Kosong O Kosong
Gau Kosong O Kosong
Peng Kosong O Kosong
Kosong O Kosong
Siew O Kosong
C Siew O Kosong
O Siew O Kosong
Gah Siew O Kosong
Kosong Siew O Kosong
Siew Siew O 

O Peng Gah Gau
Gah Peng Gah Gau
Kosong Peng Gah Gau
Siew Peng Gah Gau
Dai Peng Gah Gau
Po Peng Gah Gau
Gau Peng Gah Gau
Peng Peng Gah Gau
Peng Gah Gau
Gah Gau
C Gah Gau
O Gah Gau
Gah Gah Gau
Kosong Gah Gau
Siew Gah Gau
Dai Gah Gau
Po Gah Gau
Gau Gah Gau
Peng Gah Gau
Gah Gau
Gah Peng
C Gah Peng
C C Gah Peng
O C Gah Peng
Gah C Gah Peng
Kosong C Gah Peng
Siew C Gah Peng
Dai C Gah Peng
Po C Gah Peng
Gau C Gah Peng
Peng C Gah Peng
C Gah Peng
O Gah Peng
C O Gah Peng
O O Gah Peng
Gah O Gah Peng
Kosong O Gah Peng
Siew O Gah Peng
Dai O Gah Peng
Po O Gah Peng
Gau O Gah Peng
Peng O Gah Peng
O Gah Peng
Gah Gah Peng
C Gah Gah Peng
O Gah Gah Peng
Gah Gah Gah Peng
Kosong Gah Gah Peng
Siew Gah Gah Peng
Dai Gah Gah Peng
Po Gah Gah Peng
Gau Gah Gah Peng
Peng Gah Gah Peng
Gah Gah Peng
Kosong Gah Peng
C Kosong Gah Peng
O Kosong Gah Peng
Gah Kosong Gah Peng
Kosong Kosong Gah Peng
Siew Kosong Gah Peng
Dai Kosong Gah Peng
Po Kosong Gah Peng
Gau Kosong Gah Peng
Peng Kosong Gah Peng
Kosong Gah Peng
Siew Gah Pe

Gau Kosong Siew Gah
Peng Kosong Siew Gah
Kosong Siew Gah
Siew Siew Gah
C Siew Siew Gah
O Siew Siew Gah
Gah Siew Siew Gah
Kosong Siew Siew Gah
Siew Siew Siew Gah
Dai Siew Siew Gah
Po Siew Siew Gah
Gau Siew Siew Gah
Peng Siew Siew Gah
Siew Siew Gah
Dai Siew Gah
C Dai Siew Gah
O Dai Siew Gah
Gah Dai Siew Gah
Kosong Dai Siew Gah
Siew Dai Siew Gah
Dai Dai Siew Gah
Po Dai Siew Gah
Gau Dai Siew Gah
Peng Dai Siew Gah
Dai Siew Gah
Po Siew Gah
C Po Siew Gah
O Po Siew Gah
Gah Po Siew Gah
Kosong Po Siew Gah
Siew Po Siew Gah
Dai Po Siew Gah
Po Po Siew Gah
Gau Po Siew Gah
Peng Po Siew Gah
Po Siew Gah
Gau Siew Gah
C Gau Siew Gah
O Gau Siew Gah
Gah Gau Siew Gah
Kosong Gau Siew Gah
Siew Gau Siew Gah
Dai Gau Siew Gah
Po Gau Siew Gah
Gau Gau Siew Gah
Peng Gau Siew Gah
Gau Siew Gah
Peng Siew Gah
C Peng Siew Gah
O Peng Siew Gah
Gah Peng Siew Gah
Kosong Peng Siew Gah
Siew Peng Siew Gah
Dai Peng Siew Gah
Po Peng Siew Gah
Gau Peng Siew Gah
Peng Peng Siew Gah
Peng Siew Gah
Siew Gah
C Siew Gah
O Siew Gah
Gah Si

Gah C Dai Po
Kosong C Dai Po
Siew C Dai Po
Dai C Dai Po
Po C Dai Po
Gau C Dai Po
Peng C Dai Po
C Dai Po
O Dai Po
C O Dai Po
O O Dai Po
Gah O Dai Po
Kosong O Dai Po
Siew O Dai Po
Dai O Dai Po
Po O Dai Po
Gau O Dai Po
Peng O Dai Po
O Dai Po
Gah Dai Po
C Gah Dai Po
O Gah Dai Po
Gah Gah Dai Po
Kosong Gah Dai Po
Siew Gah Dai Po
Dai Gah Dai Po
Po Gah Dai Po
Gau Gah Dai Po
Peng Gah Dai Po
Gah Dai Po
Kosong Dai Po
C Kosong Dai Po
O Kosong Dai Po
Gah Kosong Dai Po
Kosong Kosong Dai Po
Siew Kosong Dai Po
Dai Kosong Dai Po
Po Kosong Dai Po
Gau Kosong Dai Po
Peng Kosong Dai Po
Kosong Dai Po
Siew Dai Po
C Siew Dai Po
O Siew Dai Po
Gah Siew Dai Po
Kosong Siew Dai Po
Siew Siew Dai Po
Dai Siew Dai Po
Po Siew Dai Po
Gau Siew Dai Po
Peng Siew Dai Po
Siew Dai Po
Dai Dai Po
C Dai Dai Po
O Dai Dai Po
Gah Dai Dai Po
Kosong Dai Dai Po
Siew Dai Dai Po
Dai Dai Dai Po
Po Dai Dai Po
Gau Dai Dai Po
Peng Dai Dai Po
Dai Dai Po
Po Dai Po
C Po Dai Po
O Po Dai Po
Gah Po Dai Po
Kosong Po Dai Po
Siew Po Dai Po
Dai Po Da

O Siew Gau O
Gah Siew Gau O
Kosong Siew Gau O
Siew Siew Gau O
Dai Siew Gau O
Po Siew Gau O
Gau Siew Gau O
Peng Siew Gau O
Siew Gau O
Dai Gau O
C Dai Gau O
O Dai Gau O
Gah Dai Gau O
Kosong Dai Gau O
Siew Dai Gau O
Dai Dai Gau O
Po Dai Gau O
Gau Dai Gau O
Peng Dai Gau O
Dai Gau O
Po Gau O
C Po Gau O
O Po Gau O
Gah Po Gau O
Kosong Po Gau O
Siew Po Gau O
Dai Po Gau O
Po Po Gau O
Gau Po Gau O
Peng Po Gau O
Po Gau O
Gau Gau O
C Gau Gau O
O Gau Gau O
Gah Gau Gau O
Kosong Gau Gau O
Siew Gau Gau O
Dai Gau Gau O
Po Gau Gau O
Gau Gau Gau O
Peng Gau Gau O
Gau Gau O
Peng Gau O
C Peng Gau O
O Peng Gau O
Gah Peng Gau O
Kosong Peng Gau O
Siew Peng Gau O
Dai Peng Gau O
Po Peng Gau O
Gau Peng Gau O
Peng Peng Gau O
Peng Gau O
Gau O
C Gau O
O Gau O
Gah Gau O
Kosong Gau O
Siew Gau O
Dai Gau O
Po Gau O
Gau Gau O
Peng Gau O
Gau O
Gau Gah
C Gau Gah
C C Gau Gah
O C Gau Gah
Gah C Gau Gah
Kosong C Gau Gah
Siew C Gau Gah
Dai C Gau Gah
Po C Gau Gah
Gau C Gau Gah
Peng C Gau Gah
C Gau Gah
O Gau Gah
C O Gau Gah
O O G

Dai C C
Po C C
Gau C C
Peng C C
C C
O C
C O C
O O C
Gah O C
Kosong O C
Siew O C
Dai O C
Po O C
Gau O C
Peng O C
O C
Gah C
C Gah C
O Gah C
Gah Gah C
Kosong Gah C
Siew Gah C
Dai Gah C
Po Gah C
Gau Gah C
Peng Gah C
Gah C
Kosong C
C Kosong C
O Kosong C
Gah Kosong C
Kosong Kosong C
Siew Kosong C
Dai Kosong C
Po Kosong C
Gau Kosong C
Peng Kosong C
Kosong C
Siew C
C Siew C
O Siew C
Gah Siew C
Kosong Siew C
Siew Siew C
Dai Siew C
Po Siew C
Gau Siew C
Peng Siew C
Siew C
Dai C
C Dai C
O Dai C
Gah Dai C
Kosong Dai C
Siew Dai C
Dai Dai C
Po Dai C
Gau Dai C
Peng Dai C
Dai C
Po C
C Po C
O Po C
Gah Po C
Kosong Po C
Siew Po C
Dai Po C
Po Po C
Gau Po C
Peng Po C
Po C
Gau C
C Gau C
O Gau C
Gah Gau C
Kosong Gau C
Siew Gau C
Dai Gau C
Po Gau C
Gau Gau C
Peng Gau C
Gau C
Peng C
C Peng C
O Peng C
Gah Peng C
Kosong Peng C
Siew Peng C
Dai Peng C
Po Peng C
Gau Peng C
Peng Peng C
Peng C
C
C C
O C
Gah C
Kosong C
Siew C
Dai C
Po C
Gau C
Peng C
C
O
C O
C C O
O C O
Gah C O
Kosong C O
Siew C O
Dai C O
Po C O
Gau C O

In [53]:
from itertools import product
x = 'apple orange pair None'.split()
for i, j in product(x, x):
    i = '' if i == 'None' else i
    j = '' if j == 'None' else j
    y = i + ' ' + j
    y = y.strip()
    print(y)
    for k, l in product(x, [y]):
        k = '' if k == 'None' else k
        l = '' if l == 'None' else l
        z = k + ' ' + l
        z = z.strip()
        print(z)

apple apple
apple apple apple
orange apple apple
pair apple apple
apple apple
apple orange
apple apple orange
orange apple orange
pair apple orange
apple orange
apple pair
apple apple pair
orange apple pair
pair apple pair
apple pair
apple
apple apple
orange apple
pair apple
apple
orange apple
apple orange apple
orange orange apple
pair orange apple
orange apple
orange orange
apple orange orange
orange orange orange
pair orange orange
orange orange
orange pair
apple orange pair
orange orange pair
pair orange pair
orange pair
orange
apple orange
orange orange
pair orange
orange
pair apple
apple pair apple
orange pair apple
pair pair apple
pair apple
pair orange
apple pair orange
orange pair orange
pair pair orange
pair orange
pair pair
apple pair pair
orange pair pair
pair pair pair
pair pair
pair
apple pair
orange pair
pair pair
pair
apple
apple apple
orange apple
pair apple
apple
orange
apple orange
orange orange
pair orange
orange
pair
apple pair
orange pair
pair pair
pair

apple
ora

In [57]:
[y for y in product(x, repeat=4) if len(set(y)) == len(y)]

[('C', 'O', 'Gah', 'Kosong'),
 ('C', 'O', 'Gah', 'Siew'),
 ('C', 'O', 'Gah', 'Dai'),
 ('C', 'O', 'Gah', 'Po'),
 ('C', 'O', 'Gah', 'Gau'),
 ('C', 'O', 'Gah', 'Peng'),
 ('C', 'O', 'Gah', 'None'),
 ('C', 'O', 'Kosong', 'Gah'),
 ('C', 'O', 'Kosong', 'Siew'),
 ('C', 'O', 'Kosong', 'Dai'),
 ('C', 'O', 'Kosong', 'Po'),
 ('C', 'O', 'Kosong', 'Gau'),
 ('C', 'O', 'Kosong', 'Peng'),
 ('C', 'O', 'Kosong', 'None'),
 ('C', 'O', 'Siew', 'Gah'),
 ('C', 'O', 'Siew', 'Kosong'),
 ('C', 'O', 'Siew', 'Dai'),
 ('C', 'O', 'Siew', 'Po'),
 ('C', 'O', 'Siew', 'Gau'),
 ('C', 'O', 'Siew', 'Peng'),
 ('C', 'O', 'Siew', 'None'),
 ('C', 'O', 'Dai', 'Gah'),
 ('C', 'O', 'Dai', 'Kosong'),
 ('C', 'O', 'Dai', 'Siew'),
 ('C', 'O', 'Dai', 'Po'),
 ('C', 'O', 'Dai', 'Gau'),
 ('C', 'O', 'Dai', 'Peng'),
 ('C', 'O', 'Dai', 'None'),
 ('C', 'O', 'Po', 'Gah'),
 ('C', 'O', 'Po', 'Kosong'),
 ('C', 'O', 'Po', 'Siew'),
 ('C', 'O', 'Po', 'Dai'),
 ('C', 'O', 'Po', 'Gau'),
 ('C', 'O', 'Po', 'Peng'),
 ('C', 'O', 'Po', 'None'),
 ('C', 'O', 